In [ ]:
import pandas as pd
  
#sheet별 테이블 처리
xl = pd.ExcelFile("Partial_record.xlsx")
xl.sheet_names
df1 = xl.parse("value")
df2 = xl.parse("real_record")
  
#value가 긴 값 처리
pd.set_option('display.max_colwidth', 100)
df2.head()
  
#파싱 대상
#수동 파싱
origin_string = df2["record_str"]
  
game = []
ab = []
hit = []
hit2b = []
hit3b = []
hr = []
bb = []
hbp = []
sb = []
rbi = []
r = []
sf = []
  
for object in origin_string:
    object = object.replace("{", "")
    object = object.replace("}", "")
    object = object.replace('"', "")
    list = object.split(",")
    record_list = []
    for a in list:
        index = a.index(":")
        record_list.append(a[index+1:])
     
    game.append(record_list[0])
    ab.append(record_list[1])
    hit.append(record_list[2])
    hit2b.append(record_list[3])
    hit3b.append(record_list[4])
    hr.append(record_list[5])
    bb.append(record_list[6])
    hbp.append(record_list[7])
    sb.append(record_list[8])
    rbi.append(record_list[9])
    r.append(record_list[10])
    sf.append(record_list[11])
  
#칼럼 추가
df2["game"] = game
df2["ab"] = ab
df2["hit"] = hit
df2["hit2b"] = hit2b
df2["hit3b"] = hit3b
df2["hr"] = hr
df2["bb"] = bb
df2["hbp"] = hbp
df2["sb"] = sb
df2["rbi"] = rbi
df2["r"] = r
df2["sf"] = sf
  
columns = df2.columns[[0,1,5,6,7,8,9,10,11,12,13,14,15,16]]
new_df2 = df2[columns]
new_df2 = new_df2.astype({"date_id": str, "person_id": str})
new_df2["index"] = new_df2["date_id"] + new_df2["person_id"]
   
df1 = df1.astype({"date_id": str, "person_id": str})
df1["index"] = df1["date_id"] + df1["person_id"]
#index 생성 후 key값으로 merge
df = pd.merge(df1, new_df2, on = "index")
  
df = df.drop(["date_id_y", "person_id_y"], 1)
df = df.drop("index", 1)
df = df.rename(columns = {"date_id_x": "date_id", "person_id_x": "person_id", "hit_1b":"lp_1b", "hit_2b":"lp_2b", "hit_3b":"lp_3b", "hr_x":"lp_hr", "bb_x":"lp_bb","sb_x":"lp:sb"})
df = df.rename(columns = {"hit_1b_yn":"yn_1b", "hit_2b_yn":"yn_2b", "hit_3b_yn":"yn_3b", "hr_yn":"yn_hr", "bb_yn":"yn_bb", "sb_yn":"yn_sb"})
df = df.rename(columns = {"lp:sb":"lp_sb", "hit2b":"2b","hit3b":"3b", "hr_y":"hr","bb_y":"bb", "sb_y":"sb"})
  
df = df.astype({"game": int, "ab": int, "hit":int, "2b":int, "3b": int, "hr":int, "bb":int, "hbp":int, "sb":int, "rbi":int, "r":int, "sf":int})
df["1b"] = df["hit"] - df["2b"] - df["3b"] - df["hr"]
  
#칼럼 순서 변경
column_list = df.columns.to_list()
   
list = []
list.append(column_list[-1])
new_column_list = column_list[:18] + list + column_list[18:-1]
df = df[new_column_list]

#index 생성 후 key값으로 merge
df = pd.merge(df1, new_df2, on = "index")
  
#데이터 전처리
#1루타 값 생성
df = df.astype({"game": int, "ab": int, "hit":int, "2b":int, "3b": int, "hr":int, "bb":int, "hbp":int, "sb":int, "rbi":int, "r":int, "sf":int})
df["1b"] = df["hit"] - df["2b"] - df["3b"] - df["hr"]
  
#칼럼 순서 변경
column_list = df.columns.to_list()
  
list = []
list.append(column_list[-1])
new_column_list = column_list[:18] + list + column_list[18:-1]
df = df[new_column_list]
  
#1차 저장
df.to_excel("preprocessing.xlsx")

import pandas as pd
  
file = pd.ExcelFile("preprocessing.xlsx")
df = file.parse("Sheet1")
df.head()
  
df = df.drop("Unnamed: 0", 1)
  
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.max_columns", 1000)
pd.set_option('display.float_format', '{:.0f}'.format)
  
df["pa"] = df["ab"] + df["hbp"] + df["bb"] + df["sf"]
  
constant_1b = 2
df["exp_1b"] = (df["1b"] / df["pa"]) * df["lp_1b"] * constant_1b
  
constant_2b = 0.8
df["exp_2b"] = (df["2b"] / df["pa"]) * df["lp_2b"] * constant_2b
  
constant_3b = 0.3
df["exp_3b"] = (df["3b"] / df["pa"]) * df["lp_3b"] * constant_3b
  
constant_hr = 0.6
df["exp_hr"] = (df["hr"] / df["pa"]) * df["lp_hr"] * constant_hr
  
constant_sb = 0.4
df["exp_sb"] = ((df["1b"] + df["bb"])/df["pa"]) * (df["sb"]/(df["1b"]+df["bb"])/1.5) * df["lp_sb"] * constant_sb
  
constant_bb = 1
df["exp_bb"] = (df["bb"] / df["pa"]) * df["lp_bb"] * constant_bb
  
df.fillna(0, inplace = True)
  
team_list = df["team_code"].unique()
date_list = df["date_id"].unique()
  
exp_columns = ["exp_1b","exp_2b","exp_3b","exp_hr","exp_bb","exp_sb"]
match = {"exp_1b":"lp_1b", "exp_2b":"lp_2b", "exp_3b":"lp_3b", "exp_hr":"lp_hr", "exp_sb": "lp_sb", "exp_bb":"lp_bb"}
result = {}
for date in date_list:
    daily_result = {}
    for team in team_list:
        instant_df = df[(df["date_id"] == date) & (df["team_code"] == team)][exp_columns]
        instant_df["best"] = instant_df.max(axis = 1)
        instant_df.sort_values(["best"], ascending = False, inplace = True)
        pick = instant_df.head(4).idxmax(axis = 1)
     
        pick_lp = []
        for index in pick.index:
            lp = df.iloc[index][match[pick[index]]]
            pick_lp.append(lp)
            sum_lp = sum(pick_lp)*8
        daily_result[team] = sum_lp
         
    max_team = max(daily_result.keys(), key = (lambda k: daily_result[k]))
    instant_df = df[(df["date_id"] == date) & (df["team_code"] == max_team)][exp_columns]
    instant_df["best"] = instant_df.max(axis = 1)
    instant_df.sort_values(["best"], ascending = False, inplace = True)
    exp_record = instant_df.head(4).idxmax(axis = 1)
     
    pick_record = []
    for record in exp_record:
        pick_record.append(match[record])
     
    pick_person = []
    for index in exp_record.index:
        person = df.iloc[index]["person_id"]
        pick_person.append(person)
         
    daily_pick = dict(zip(pick_person, pick_record))
    result[date] = daily_pick
     
print(result)

answer_match = {"lp_1b":"yn_1b", "lp_2b":"yn_2b", "lp_3b":"yn_3b", "lp":"yn_hr", "lp_sb": "yn_sb", "lp_bb":"yn_bb"}
get_point = []
for date in date_list:
    pick_person_id = result[date].keys()
    daily_get_point = []
     
    for person_id in pick_person_id:
        daily_result = result[date]
        pick_record = daily_result[person_id]
        check_record = answer_match[pick_record]
        if df[(df["date_id"] == date) & (df["person_id"] == person_id)][check_record].values == "Y":
            get_lp_df = df[(df["date_id"] == date) & (df["person_id"] == person_id)][pick_record]
            get_lp = get_lp_df.iloc[0]
            daily_get_point.append(get_lp)
        else:
            daily_get_point.append(0)
             
    if sum(i > 0 for i in daily_get_point) == 4:
        daily_get_point = sum(daily_get_point) * 8
    elif sum(i > 0 for i in daily_get_point) == 3:
        daily_get_point = sum(daily_get_point) * 4
    elif sum(i > 0 for i in daily_get_point) == 2:
        daily_get_point = sum(daily_get_point) * 2
    elif sum(i > 0 for i in daily_get_point) == 1:
        daily_get_point = sum(daily_get_point) * 1
    else:
        daily_get_point = sum(daily_get_point) * 0
     
    get_point.append(daily_get_point)
print(sum(get_point))

import pandas as pd
import numpy as np
    
#sheet별 테이블 처리
xl = pd.ExcelFile("2018_live_player.xlsx")
xl.sheet_names
df1 = xl.parse("value")
df2 = xl.parse("real_record")
    
#value가 긴 값 처리
pd.set_option('display.max_colwidth', 100)
df2.head()
#파싱 대상
#수동 파싱
origin_string = df2["record_str"]
    
game = []
ab = []
hit = []
hit2b = []
hit3b = []
hr = []
bb = []
hbp = []
sb = []
rbi = []
r = []
sf = []
    
for object in origin_string:
    object = object.replace("{", "")
    object = object.replace("}", "")
    object = object.replace('"', "")
    list = object.split(",")
    record_list = []
    for a in list:
        index = a.index(":")
        record_list.append(a[index+1:])
       
    game.append(record_list[0])
    ab.append(record_list[1])
    hit.append(record_list[2])
    hit2b.append(record_list[3])
    hit3b.append(record_list[4])
    hr.append(record_list[5])
    bb.append(record_list[6])
    hbp.append(record_list[7])
    sb.append(record_list[8])
    rbi.append(record_list[9])
    r.append(record_list[10])
    sf.append(record_list[11])
    
#칼럼 추가
df2["game"] = game
df2["ab"] = ab
df2["hit"] = hit
df2["hit2b"] = hit2b
df2["hit3b"] = hit3b
df2["hr"] = hr
df2["bb"] = bb
df2["hbp"] = hbp
df2["sb"] = sb
df2["rbi"] = rbi
df2["r"] = r
df2["sf"] = sf
    
columns = df2.columns[[0,1,5,6,7,8,9,10,11,12,13,14,15,16]]
new_df2 = df2[columns]
new_df2 = new_df2.astype({"date_id": str, "person_id": str})
new_df2["index"] = new_df2["date_id"] + new_df2["person_id"]
     
df1 = df1.astype({"date_id": str, "person_id": str})
df1["index"] = df1["date_id"] + df1["person_id"]
#index 생성 후 key값으로 merge
df = pd.merge(df1, new_df2, on = "index")
    
df = df.drop(["date_id_y", "person_id_y"], 1)
df = df.drop("index", 1)
df = df.rename(columns = {"date_id_x": "date_id", "person_id_x": "person_id", "hit_1b":"lp_1b", "hit_2b":"lp_2b", "hit_3b":"lp_3b", "hr_x":"lp_hr", "bb_x":"lp_bb","sb_x":"lp:sb"})
df = df.rename(columns = {"hit_1b_yn":"yn_1b", "hit_2b_yn":"yn_2b", "hit_3b_yn":"yn_3b", "hr_yn":"yn_hr", "bb_yn":"yn_bb", "sb_yn":"yn_sb"})
df = df.rename(columns = {"lp:sb":"lp_sb", "hit2b":"2b","hit3b":"3b", "hr_y":"hr","bb_y":"bb", "sb_y":"sb"})
    
df = df.astype({"game": int, "ab": int, "hit":int, "2b":int, "3b": int, "hr":int, "bb":int, "hbp":int, "sb":int, "rbi":int, "r":int, "sf":int})
df["1b"] = df["hit"] - df["2b"] - df["3b"] - df["hr"]
    
#칼럼 순서 변경
column_list = df.columns.to_list()
     
list = []
list.append(column_list[-1])
new_column_list = column_list[:18] + list + column_list[18:-1]
df = df[new_column_list]
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.max_columns", 1000)
pd.set_option('display.float_format', '{:.0f}'.format)
    
df["pa"] = df["ab"] + df["hbp"] + df["bb"] + df["sf"]
for constant_1b in np.arange(0.1, 4.1, 0.1):
    df["exp_1b"] = (df["1b"] / df["pa"]) * df["lp_1b"] * constant_1b
    constant_2b = 0.65
    df["exp_2b"] = (df["2b"] / df["pa"]) * df["lp_2b"] * constant_2b
    constant_3b = 0.3
    df["exp_3b"] = (df["3b"] / df["pa"]) * df["lp_3b"] * constant_3b
    constant_hr = 0.6
    df["exp_hr"] = (df["hr"] / df["pa"]) * df["lp_hr"] * constant_hr
    constant_sb = 0.4
    df["exp_sb"] = ((df["1b"] + df["bb"])/df["pa"]) * (df["sb"]/(df["1b"]+df["bb"])/1.5) * df["lp_sb"] * constant_sb
    constant_bb = 1
    df["exp_bb"] = (df["bb"] / df["pa"]) * df["lp_bb"] * constant_bb
    df.fillna(0, inplace = True)
    team_list = df["team_code"].unique()
    date_list = df["date_id"].unique()
    exp_columns = ["exp_1b","exp_2b","exp_3b","exp_hr","exp_bb","exp_sb"]
    match = {"exp_1b":"lp_1b", "exp_2b":"lp_2b", "exp_3b":"lp_3b", "exp_hr":"lp_hr", "exp_sb": "lp_sb", "exp_bb":"lp_bb"}
    result = {}
    for date in date_list:
        daily_result = {}
        for team in team_list:
            instant_df = df[(df["date_id"] == date) & (df["team_code"] == team) & (df["pa"] >= 30)][exp_columns]
            instant_df["best"] = instant_df.max(axis = 1)
            instant_df.sort_values(["best"], ascending = False, inplace = True)
            pick = instant_df.head(4).idxmax(axis = 1)
            pick_lp = []
            for index in pick.index:
                lp = df.iloc[index][match[pick[index]]]
                pick_lp.append(lp)
                sum_lp = sum(pick_lp)
            daily_result[team] = sum_lp
        max_team = max(daily_result.keys(), key = (lambda k: daily_result[k]))
        instant_df = df[(df["date_id"] == date) & (df["team_code"] == max_team)][exp_columns]
        instant_df["best"] = instant_df.max(axis = 1)
        instant_df.sort_values(["best"], ascending = False, inplace = True)
        exp_record = instant_df.head(4).idxmax(axis = 1)
        pick_record = []
        for record in exp_record:
            pick_record.append(match[record])
        pick_person = []
        for index in exp_record.index:
            person = df.iloc[index]["person_id"]
            pick_person.append(person)
        daily_pick = dict(zip(pick_person, pick_record))
        result[date] = daily_pick
    answer_match = {"lp_1b":"yn_1b", "lp_2b":"yn_2b", "lp_3b":"yn_3b", "lp_hr":"yn_hr", "lp_sb":"yn_sb", "lp_bb":"yn_bb"}
    get_point = []
    for date in date_list:
        pick_person_id = result[date].keys()
        daily_get_point = []
        for person_id in pick_person_id:
            daily_result = result[date]
            pick_record = daily_result[person_id]
            check_record = answer_match[pick_record]
            if df[(df["date_id"] == date) & (df["person_id"] == person_id)][check_record].values == "Y":
                get_lp_df = df[(df["date_id"] == date) & (df["person_id"] == person_id)][pick_record]
                get_lp = get_lp_df.iloc[0]
                daily_get_point.append(get_lp)
            else:
                daily_get_point.append(0)
        if sum(i > 0 for i in daily_get_point) == 4:
            daily_get_point = sum(daily_get_point) * 8
        elif sum(i > 0 for i in daily_get_point) == 3:
            daily_get_point = sum(daily_get_point) * 4
        elif sum(i > 0 for i in daily_get_point) == 2:
            daily_get_point = sum(daily_get_point) * 2
        elif sum(i > 0 for i in daily_get_point) == 1:
            daily_get_point = sum(daily_get_point) * 1
        else:
            daily_get_point = sum(daily_get_point) * 0
        get_point.append(daily_get_point)
        total_point = sum(get_point)
    print("constant_1b: {}, get_point: {}".format(constant_1b, total_point))